In [44]:
import numpy as np
import pandas as pd
import math

## Load Dataset

In [93]:
'''
COLUMN NAME - INDEX
gross           8
movie_title    11
language       19
country        20
'''
def loadDataset(filename):
    dataset = pd.read_csv(filename, delimiter=',')
    return np.array(dataset)

In [94]:
# DEBUG
datasetTest = loadDataset('movie_metadata.csv')
datasetTest

array([['Color', 'James Cameron', 723.0, ..., 7.9, 1.78, 33000],
       ['Color', 'Gore Verbinski', 302.0, ..., 7.1, 2.35, 0],
       ['Color', 'Sam Mendes', 602.0, ..., 6.8, 2.35, 85000],
       ...,
       ['Color', 'Benjamin Roberds', 13.0, ..., 6.3, nan, 16],
       ['Color', 'Daniel Hsia', 14.0, ..., 6.3, 2.35, 660],
       ['Color', 'Jon Gunn', 43.0, ..., 6.6, 1.85, 456]], dtype=object)

In [103]:
clearingDatasetQ1(datasetTest)

[array([760505847.0, 'Avatar\xa0'], dtype=object),
 array([309404152.0, "Pirates of the Caribbean: At World's End\xa0"],
       dtype=object),
 array([200074175.0, 'Spectre\xa0'], dtype=object),
 array([448130642.0, 'The Dark Knight Rises\xa0'], dtype=object),
 array([73058679.0, 'John Carter\xa0'], dtype=object),
 array([336530303.0, 'Spider-Man 3\xa0'], dtype=object),
 array([200807262.0, 'Tangled\xa0'], dtype=object),
 array([458991599.0, 'Avengers: Age of Ultron\xa0'], dtype=object),
 array([301956980.0, 'Harry Potter and the Half-Blood Prince\xa0'],
       dtype=object),
 array([330249062.0, 'Batman v Superman: Dawn of Justice\xa0'],
       dtype=object),
 array([200069408.0, 'Superman Returns\xa0'], dtype=object),
 array([168368427.0, 'Quantum of Solace\xa0'], dtype=object),
 array([423032628.0, "Pirates of the Caribbean: Dead Man's Chest\xa0"],
       dtype=object),
 array([89289910.0, 'The Lone Ranger\xa0'], dtype=object),
 array([291021565.0, 'Man of Steel\xa0'], dtype=object)

## Clearing Dataset

In [101]:
def clearingDatasetQ1(dataset):
    subset = dataset[:, [8, 11]]
    new_dataset = list()
    for record in subset:
        if not(math.isnan(record[0])):
            new_dataset.append(record)
    return new_dataset

## Possible values

In [96]:
def possibleValues(dataset):
    indexRecord = 0
    titleUniqueMovies = list()
    cleanDataset = clearingDatasetQ1(dataset)
    removingReplicates = np.copy(cleanDataset)
    
    for record in cleanDataset:
        if record[1] not in titleUniqueMovies:
            titleUniqueMovies.append(record[1])
        else:
            removingReplicates = np.delete(removingReplicates, indexRecord, 0)
            indexRecord -= 1
            
        indexRecord += 1
       
    return np.array(removingReplicates)

In [97]:
# DEBUG
newDataset = possibleValues(datasetTest)

## Score Function

In [98]:
def cartesianProduct(attributeA, attributeB):
    return np.transpose([np.tile(attributeA, len(attributeB)),
                         np.repeat(attributeB, len(attributeA))])

In [99]:
def scoreFunctionQ1(dataset):
    grossMovies = dataset[:, 0]
    titleMovies = dataset[:, 1]
    
    score = list()
    cartesianProductMoviesGross = cartesianProduct(grossMovies, titleMovies)
    
    for record in possibleValues(dataset):
        if record in cartesianProductMoviesGross:
            score.append(record[0])
        else:
            score.append(0)
    return score

In [100]:
scoreFunctionQ1(newDataset)

IndexError: index 8 is out of bounds for axis 1 with size 2

In [20]:
def main(filename, budgets):
    dataset = loadDataset(filename)

In [19]:
main('movie_metadata.csv', [])